In [ ]:
!pip install google-cloud-core
!pip install google-cloud-bigtable

In [2]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family
import pandas as pd


**1. Carga de los datos**

In [81]:
plant1_gen_data = pd.read_csv('/content/Plant_1_Generation_Data.csv')
plant1_weather = pd.read_csv('/content/Plant_1_Weather_Sensor_Data.csv')
plant2_gen_data = pd.read_csv('/content/Plant_2_Generation_Data.csv')
plant2_weather = pd.read_csv('/content/Plant_2_Weather_Sensor_Data.csv')

**2. Conexión a la instacia de BigTable**

In [131]:
INSTANCE_ID = 'table-1'
client = bigtable.Client.from_service_account_json('/content/unbosque-service-account.json', admin = True)
print('Connecting to database instance')
instances = client.list_instances()
print(instances[0])
for i in instances:
  for j in i:
    print(j.display_name)

instance = client.instance(INSTANCE_ID)
instance.exists()
#instance.create()



Connecting to database instance
table-1


True

**Nombre de la tabla a crear**

In [116]:
TABLE_ID = 'Workshop4_RS'
TABLE_ID

'Workshop4_RS'

**3. Crear la Tabla**

In [133]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the Workshop4_RS table
ERROR: Table Workshop4_RS already exists


**4. Creación de las colecciones**

In [ ]:
print('Creating all family columns')

plant_cfId = 'plant'
plant_cf = table.column_family(plant_cfId)
plant_cf.create()

power_cfId = 'power'
power_cf = table.column_family(power_cfId)
power_cf.create()

weather_cfId = 'weather'
weather_cf = table.column_family(weather_cfId)
weather_cf.create()

In [59]:
for i in plant1_gen_data['DATE_TIME'].index:
  plant1_gen_data['DATE_TIME'][i] = dt.strptime(plant1_gen_data['DATE_TIME'][i], '%d-%m-%Y %H:%M')
for i in plant2_gen_data['DATE_TIME'].index:
  plant2_gen_data['DATE_TIME'][i] = dt.strptime(plant2_gen_data['DATE_TIME'][i], '%Y-%m-%d %H:%M:%S')
for i in plant1_weather['DATE_TIME'].index:
  plant1_weather['DATE_TIME'][i] = dt.strptime(plant1_weather['DATE_TIME'][i], '%Y-%m-%d %H:%M:%S')
for i in plant2_weather['DATE_TIME'].index:
  plant2_weather['DATE_TIME'][i] = dt.strptime(plant2_weather['DATE_TIME'][i], '%Y-%m-%d %H:%M:%S')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: Setti

**5. Definición que inserta los datos en la tabla**

In [ ]:
def insert_data(data, data2):
  rows = []
  row_key = str(data['PLANT_ID'])[0].encode()
  print(row_key)
  row = table.direct_row(row_key)
    
  for d in data.iterrows():
    row.set_cell(plant_cfId, 'plant_id'.encode(),d[1]['PLANT_ID'])
    row.set_cell(power_cfId, 'Source_id'.encode(),d[1]['SOURCE_KEY'])
    row.set_cell(power_cfId,'DC_POWER'.encode(), str(d[1]['DC_POWER']))
    row.set_cell(power_cfId,'AC_POWER'.encode(), str(d[1]['AC_POWER']))
    row.set_cell(power_cfId,'DAILY_YIELD'.encode(), str(d[1]['DAILY_YIELD']))
    row.set_cell(power_cfId,'TOTAL_YIELD'.encode(), str(d[1]['TOTAL_YIELD']))
    row.set_cell(power_cfId,'DATE_TIME'.encode(), d[1]['DATE_TIME'])
  
  for c in data2.iterrows():
    row.set_cell(weather_cfId, 'SOURCE_KEY'.encode(),c[1]['SOURCE_KEY'])
    row.set_cell(weather_cfId,'AMBIENT_TEMPERATURE'.encode(), str(c[1]['AMBIENT_TEMPERATURE']))
    row.set_cell(weather_cfId,'MODULE_TEMPERATURE'.encode(), str(c[1]['MODULE_TEMPERATURE']))
    row.set_cell(weather_cfId,'IRRADIATION'.encode(), str(c[1]['IRRADIATION']))
    row.set_cell(weather_cfId,'DATE_TIME'.encode(), c[1]['DATE_TIME'])

  rows.append(row)

  table.mutate_rows(rows)

insert_data(plant1_gen_data, plant1_weather)
insert_data(plant2_gen_data, plant2_weather)

In [127]:
print('Deleting the {} table.'.format(TABLE_ID))
table.delete()

Deleting the Workshop4_RS table.


In [123]:
print('Getting all orders')

rows = table.read_rows()
for row in rows:
    print_row(row)

Getting all orders


**6. Definiciones que consultan la tabla por filtros**

In [ ]:
def leer_data_fechas(fecha, planta):
  row_set = RowSet()
  row_set.add_row_key(planta.encode())

  rows = table.read_rows(row_set=row_set)
  for row in rows:
    if(row['DATE_TIME']>= fecha ):
        print_row(row)

def leer_data_fechas_planta(fecha, planta, identificador):
  row_set = RowSet()
  row_set.add_row_key(planta.encode())

  rows = table.read_rows(row_set=row_set)
  for row in rows:
    if(row['DATE_TIME']>= fecha and row['SOURCE_KEY'] == identificador):
        print_row(row)

leer_data_fechas_planta('','4135001','adLQvlD726eNBSB')